In [ ]:
# !pip install -r requirements.txt

In [ ]:
import os
from os.path import exists
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.functional import log_softmax, pad
from torch.autograd import Variable
import math
import copy
import time
from torch.optim.lr_scheduler import LambdaLR
import pandas as pd
import altair as alt
from torchtext.data.functional import to_map_style_dataset
from torch.utils.data import DataLoader
from torchtext.vocab import build_vocab_from_iterator
import torchtext.datasets as datasets
import spacy
import GPUtil
import warnings
from torch.utils.data.distributed import DistributedSampler
import torch.distributed as dist
import torch.multiprocessing as mp
from torch.nn.parallel import DistributedDataParallel as DDP


# Set to False to skip notebook execution (e.g. for debugging)
warnings.filterwarnings("ignore")
RUN_EXAMPLES = True

/opt/anaconda3/envs/ml-playground/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Some convenience helper functions used throughout the notebook

def is_interactive_notebook():
    return __name__ == "__main__"


def show_example(fn, args=[]):
    if __name__ == "__main__" and RUN_EXAMPLES:
        return fn(*args)


def execute_example(fn, args=[]):
    if __name__ == "__main__" and RUN_EXAMPLES:
        fn(*args)


class DummyOptimizer(torch.optim.Optimizer):
    def __init__(self):
        self.param_groups = [{"lr": 0}]
        None

    def step(self):
        None

    def zero_grad(self, set_to_none=False):
        None


class DummyScheduler:
    def step(self):
        None

# Transformer Architecture

In [ ]:
class EncoderDecoder(nn.Module):
    # Inherits nn.Module - define __init_ and forward methods
    def __init__(self, encoder, decoder, src_embed, tgt_embed, generator):
        # Make sure the initialization of the child class follow the parent class internal settings
        # e.g. being able to use default features for nn.Module class
        super(EncoderDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.src_embed = src_embed
        self.tgt_embed = tgt_embed
        self.generator = generator
    
    def forward(self, src, tgt, src_mask, tgt_mask):
        # Forward pass through the encoder and decoder
        # recursively call the encode and decode make PyTorch more effective
        return self.decode(self.encode(src, src_mask), src_mask, tgt, tgt_mask)
    
    def encoder(self, src, src_mask):
        return self.encoder(self.src_embed(src), src_mask)

    def decoder(self, memory, src_mask, tgt, tgt_mask):
        # masks for prevent attention to padding tokens & future tokens @ decoding time
        return self.decoder(self.tgt_embed(tgt), memory, src_mask, tgt_mask)

class Generator(nn.Module):
    # Generator acts as the final linear projection + softmax
    # map hidden layer outputs to vocabulary
    def __init__(self, d_model, vocab):
        super(Generator, self).__init__()
        self.proj = nn.Linear(d_model, vocab)
    
    def forward(self, x):
        return log_softmax(self.proj(x), dim=-1)

In [ ]:
def clones(module, N):
    # for stacking layers
    # nn.ModuleList: container module holding submodules in a list
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

## Encoder
- stack of N=6 identical layers

In [ ]:
class Encoder(nn.Module):
    # generic encoder stack of N identical layers
    def __init__(self, layer, N):
        super(Encoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)
    
    def forward(self, x, mask):
        for layer in self.layers:
            x = layer(x, mask)
        return self.norm(x)

class LayerNorm(nn.Module):
    def __init__(self, features, eps=1e-6):
        super(LayerNorm, self).__init__()
        # use a_2 and b_2 as scaling and shifting parameters
        self.a_2 = nn.Parameter(torch.ones(features))
        self.b_2 = nn.Parameter(torch.zeros(features)) 
        self.eps = eps
    
    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return 

class SublayerConnection(nn.Module):
    # layer after attention/feed-forward sublayer
    # applies layer normalization and dropout
    def __init__(self, size, dropout):
        super(SublayerConnection, self).__init__()
        self.norm = LayerNorm(size)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x, sublayer):
        # pre-norm: x + dropout(sublayer(layer_norm(x))) -> more robust and fast convergence
        # sublayer can be any function computation - attention, feed-forward, etc.
        return x + self.dropout(sublayer(self.norm(x)))

class EncoderLayer(nn.Module):
    # core encoder layer - attention + feed-forward
    def __init__(self, size, self_attn, feed_forward, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = self_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 2) # two add & norm layers
        self.size = size
    
    def forward(self, x, mask):
        # apply self-attention and feed-forward network
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, mask))
        return self.sublayer[1](x, self.feed_forward)

## Decoder
- stack of 6 identical layers

In [ ]:
class Decoder(nn.Module):
    # generic N layer decoder with masking
    def __init__(self, layer, N):
        super(Decoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)
    
    def forward(self, x, memory, src_mask, tgt_mask):
        for layer in self.layers:
            x = layer(x, memory, src_mask, tgt_mask)
        return self.norm(x)

class DecoderLayer(nn.Module):
    # core decoder layer - self-attention + encoder-decoder attention + feed-forward
    def __init__(self, size, self_attn, src_attn, feed_forward, dropout):
        super(DecoderLayer, self).__init__()
        self.size = size
        self.self_attn = self_attn
        self.src_attn = src_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 3) # three add & norm layers
    
    def forward(self, x, memory, src_mask, tgt_mask):
        m = memory # output of the final layer for encoder
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, tgt_mask)) # self-attention
        x = self.sublayer[1](x, lambda x: self.src_attn(m, m, src_mask)) # encoder-decoder attention
        return self.sublayer[2](x, self.feed_forward)

def subsequent_mask(size):
    # create decoder mask
    attn_shape = (1, size, size) # 3-D mask to be used in multi-head attention (for batch broadcasting)
    subsequent_mask = np.triu(np.ones(attn_shape), k=1).astype('uint8') # block the upper triangular part of the matrix to k
    return torch.from_numpy(subsequent_mask) == 0 # invert the mask & turn into boolean entries

## Attention

In [ ]:
def attention(query, key, value, mask=None, dropout=None):
    # compute scaled dot-product attention
    # size of q, k, v: (batch_size, num_heads, seq_len, d_model)
    d_k = query.size(-1)
    scores = torch.matmul(query, key.transpose(-2,-1)) / math.sqrt(d_k) # shape: (batch_size, num_heads, seq_len_q, seq_len_k)

    if mask is not None:
        scores = scores.masked_fill(mask == 0, -1e9) # if masked turn to -inf
    p_attn = torch.softmax(scores, dim=-1) # softmax over the key positions: for each query token, generate probability distribution over all key tokens
    if dropout is not None:
        p_attn = dropout(p_attn)
    return torch.matmul(p_attn, value), p_attn
    # shape of value: (batch_size, num_heads, seq_len_k, d_v)
    # final output shape: (batch_size, num_heads, seq_len_q, d_v)

In [ ]:
class MultiHeadedAttention(nn.Module):
    def __init__(self, h, d_model, dropout=0.1):
        super(MultiHeadedAttention, self).__init__()
        assert d_model % h == 0
        # assumption: dv = dk
        self.d_k = d_model // h
        self.h = h
        self.linears = clones(nn.Linear(d_model, d_model), 4) # 3 linear layers to transform q, k, v + 1 for output
        self.attn = None
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, query, key, value, mask=None):
        if mask is not None:
            mask = mask.unsqueeze(1)
        nbatches = query.size(0) # batch size

        # 1) do all linear projections
        # .view(-1): automatically infer the size of the first dimension
        # reshape into separated heads
        query, key, value = [l(x).view(nbatches, -1, self.h, self.d_k).transpose(1,2) for l, x in zip(self.linears, (query, key, value))]

        # 2) apply attention to all the projected vectors in batch
        x, self.attn = attention(query, key, value, mask=mask, dropout=self.dropout)

        # 3) concat using a view and apply final linear projection
        # contiguous() ensures that the tensor is stored in a contiguous chunk of memory -> for .view operation (simple reshape without changing the data)
        x = x.trannspose(1, 2).contiguous().view(nbatches, -1, self.h * self.d_k)
        return self.linears[-1](x)

## Pointwise Feed-Forward Networks

In [ ]:
class PoinsitionwiseFeedForward(nn.Module):
    # pointwise FFNN - operate on each token
    # d_moel = 512, d_ff = 2048
    def __init__(self, d_model, d_ff, dropout=0.1):
        super(PoinsitionwiseFeedForward, self).__init__()
        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x):
        return self.w_2(self.dropout(F.relu(self.w_1(x))))

## Embedding

In [ ]:
class Embeddings(nn.Module):
    def __init__(self, d_model, vocab):
        super(Embeddings, self).__init__()
        self.lut = nn.Embedding(vocab, d_model)
        self.d_model = d_model
    
    def forward(self, x):
        return self.lut(x) * math.sqrt(self.d_model)

## Positional Encoding

In [ ]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * -(math.log(10000)/d_model))

        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe) # register pe as a non-learnable buffer + stores pe as part of the model
    
    def forward(self, x):
        # Variable: use positional embedding as a non-trainable tensor
        x = x + Variable(self.pe[:,:x.size(1)], requires_grad=False)
        return self.dropout(x)


## Full Model

In [ ]:
def make_model(src_vocab, tgt_vocab, N=6, d_model=512, d_ff=2048, dropout=0.1):
    c = copy.deepcopy
    attn = MultiHeadedAttention(h, d_model)
    ff = PoinsitionwiseFeedForward(d_model, d_ff, dropout)
    position = PositionalEncoding(d_model, dropout)
    model = EncoderDecoder(
        Encoder(EncoderLayer(d_model, c(attn), c(ff), dropout), N),
        Decoder(DecoderLayer(d_model, c(attn), c(attn), c(ff), dropout), N),
        nn.Sequential(Embeddings(d_model, src_vocab), c(position)),
        nn.Sequential(Embeddings(d_model, tgt_vocab), c(position)),
        Generator(d_model, tgt_vocab)
    )

    for p in model.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform_(p) # initialize weights with Xavier uniform distribution
    return model